<a href="https://colab.research.google.com/github/hank199599/deep_learning_keras_log/blob/main/Chapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 超越序列式模型

* 單輸入模型
* 多輸入模型
* 殘差連接層(residual connections)層：透過特徵圖(張量)相加，將先前的資訊重新注入下游資料流

## [函數式API](https://keras.io/guides/functional_api/)

使用Input()方法來檢立一個張量，並將張量直接傳入層(layer)或模型(model)之中，  
取得處理後的結果張量。
```python
from keras import Input,layers

input_tensor = Input(shape=(32,)) #建立一個輸入張量
print(input_tensor.shape)

dense = layers.Dense(16,activation='relu') #建立一個Dense層，並將其想像成一個函數
output_tensor = dense(input_tensor) # 將張量輸入層函數，他會回傳經處理後的結果張亮
print(output_tensor.shape)
```

### 序列式 與 函數式 

序列式(Sequential)建立模型  
![pic 7-1](https://raw.githubusercontent.com/hank199599/deep_learning_keras_log/main/pictures/7-1.png)

In [2]:
from keras.models import Sequential,Model
from keras import layers,Input

model=Sequential()
model.add(layers.Dense(32,activation='relu',input_shape=(64,)))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(32,activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
Total params: 4,192
Trainable params: 4,192
Non-trainable params: 0
_________________________________________________________________


#### 函數式(API)建立模型
透過建立**Model物件**，


In [3]:
input_tensor = Input(shape=(64,)) # 建立一個初始張量

x = layers.Dense(32,activation='relu')(input_tensor)

y = layers.Dense(32,activation='relu')(x)

output_tensor = layers.Dense(10,activation='softmax')(y)

model = Model(input_tensor,output_tensor)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


如果用完全不相干的輸入和輸出張量去建構模型。  
因為Keras找不到相關資訊，導致執行時會生錯誤。

In [4]:
unrelated_input = Input(shape=(32,))
bad_model = model = Model(unrelated_input,output_tensor)

ValueError: ignored

在編譯、訓練或驗證此Model物件時，API的功能與序列式模型相同

In [5]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
import numpy as np

x_train = np.random.random((1000,64))
y_train = np.random.random((1000,10))

# 將訓練輸入模型進行訓練
model.fit(x_train,y_train,epochs=10,batch_size=128)
score = model.evaluate(x_train,y_train)
print(score) 


Epoch 1/10
8/8 [==============================] - 1s 3ms/step - loss: 11.8730
Epoch 2/10
8/8 [==============================] - 0s 3ms/step - loss: 12.8779
Epoch 3/10
8/8 [==============================] - 0s 3ms/step - loss: 14.5742
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 16.4277
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 19.1095
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 21.7497
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 24.9432
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 28.6142
Epoch 9/10
8/8 [==============================] - 0s 2ms/step - loss: 32.1659
Epoch 10/10
32/32 [==============================] - 0s 1ms/step - loss: 40.4471
40.447086334228516


## 多輸入模型

*[範例]*：典型的問答模型  
必須針對問題產生出答案，可透過softmax分類器針對某些事件先定義好詞彙並輸出答案  
![pic 7-3](https://raw.githubusercontent.com/hank199599/deep_learning_keras_log/main/pictures/7-3.png)


In [10]:
from keras import Model 
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,),dtype='int32',name='text')
embedded_text = layers.Embedding(text_vocabulary_size,64)(text_input)
print(embedded_text.shape)
encoded_text = layers.LSTM(32)(embedded_text)
print(encoded_text.shape)

question_input = Input(shape=(None,),dtype='int32',name='question')
embedded_question = layers.Embedding(question_vocabulary_size,32)(question_input)
print(embedded_question.shape)
encoded_question = layers.LSTM(16)(embedded_question)
print(encoded_question.shape)

concatenated = layers.concatenate([encoded_question,encoded_text],axis=1)
print(concatenated.shape)

answer = layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated)
print(answer.shape)

model = Model([text_input,question_input],answer)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
model.summary()

(None, None, 64)
(None, 32)
(None, None, 32)
(None, 16)
(None, 48)
(None, 500)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, None, 64)     640000      text[0][0]                       
_____________

### 訓練雙輸入模型的方法
1. 為模型準備Numpy陣列
2. 選擇下列兩種方式進行訓練 
①將 Numpy 陣列資料組成串列(list)做為輸入進行訓練。  
②建立一個字典(dict), 將輸入透過鍵(輸入名稱),對應到值(Numpy陣列資料)。當然,此方法只有在為輸入命名時才可用,例如上面程式中的第7行
```python
Input(., name = 'text,
```
我們就可以建立 dict:
```python
{'text': Numpy 資料}  
```
做為此輸入張量對應的 Numpy 資料。

In [16]:
import numpy as np

num_samples = 1000
max_length = 100

# 產生虛擬text資料：1000筆，每筆100個字(數字)
text = np.random.randint(1,text_vocabulary_size,size = (num_samples,max_length))
print(text.shape)

# 產生虛擬question資料：1000筆，每筆100個字(數字)
question = np.random.randint(1,question_vocabulary_size,size = (num_samples,max_length))
print(question.shape)

# 產生虛擬answer資料：1000筆，每筆100個字(數字)
answers = np.zeros(shape=(num_samples,answer_vocabulary_size),dtype='int32')
print(answers.shape)


for answer in answers:
  answer[np.random.randint(answer_vocabulary_size)]=1
print(answer.shape)

# 訓練方法1：使用list將送入資料進行訓練
model.fit([text,question],answers,epochs=10,batch_size=128)

# 訓練方法2；使用dict將送入資料進行訓練，鍵為Input層的名稱，值為Numpy的值
model.fit({'text':text,'qustion':question},answers,epochs=10,batch_size=128)


(1000, 100)
(1000, 100)
(1000, 500)
(500,)
Epoch 1/10
8/8 [==============================] - 5s 133ms/step - loss: 6.2150 - acc: 0.0027
Epoch 2/10
8/8 [==============================] - 1s 131ms/step - loss: 6.1997 - acc: 0.0909
Epoch 3/10
8/8 [==============================] - 1s 132ms/step - loss: 6.1793 - acc: 0.1425
Epoch 4/10
8/8 [==============================] - 1s 131ms/step - loss: 6.1101 - acc: 0.0031
Epoch 5/10
8/8 [==============================] - 1s 129ms/step - loss: 6.0156 - acc: 0.0123
Epoch 6/10
8/8 [==============================] - 1s 126ms/step - loss: 5.9137 - acc: 0.0093
Epoch 7/10
8/8 [==============================] - 1s 132ms/step - loss: 5.8363 - acc: 0.0109
Epoch 8/10
8/8 [==============================] - 1s 134ms/step - loss: 5.7399 - acc: 0.0157
Epoch 9/10
8/8 [==============================] - 1s 129ms/step - loss: 5.6316 - acc: 0.0201
Epoch 10/10
8/8 [==============================] - 1s 128ms/step - loss: 5.5811 - acc: 0.0308
Epoch 1/10


ValueError: ignored